In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

In [3]:
from pyxirr import xirr


def Irr(date, currency, asset_id=None, broker_id_list=None, start_date=''):
    
    # Calculate portfolio value
    portfolio_value = calculate_portfolio_value(date, currency, asset_id, broker_id_list)

    if portfolio_value < 0:
        return 'N/R'
    
    cash_flows = []
    transaction_dates = []

    # Collect cash flows and transaction dates for the portfolio
    transactions = Transactions.objects.filter(date__lte=date, security_id=asset_id)
    
    # if asset_id:
    #     transactions = transactions.filter(security_id=asset_id)

    if broker_id_list:
        transactions = transactions.filter(broker_id__in=broker_id_list)

    if start_date:
        transactions = transactions.filter(date__gte=start_date)

    # print(f"utils.py. line 123. Transactions: {transactions}")

    # Calculate cash flows and transaction dates
    for transaction in transactions:
        print(f"utils.py. line 127. Transaction details: {transaction.quantity}")
        fx_rate = FX.get_rate(transaction.currency.upper(), currency, transaction.date)['FX']

        if transaction.type == 'Cash in' or transaction.type == 'Cash out':
            cash_flow = -transaction.cash_flow
        else:
            cash_flow = transaction.cash_flow or (-transaction.quantity * transaction.price + (transaction.commission or 0))
        
        cash_flows.append(round(cash_flow * fx_rate, 2))
        transaction_dates.append(transaction.date)

    # Calculate start portfolio value if provided
    if start_date:
        start_portfolio_value = calculate_portfolio_value(start_date, currency, asset_id, broker_id_list)
        cash_flows.insert(0, -start_portfolio_value)
        transaction_dates.insert(0, start_date)

    # Calculate portfolio value
    portfolio_value = calculate_portfolio_value(date, currency, asset_id, broker_id_list)

    # Append end portfolio value to cash flows and dates
    cash_flows.append(portfolio_value)
    transaction_dates.append(date)

    print(f"utils. line 146. Cash flows: {cash_flows}, Transaction dates: {transaction_dates}")

    try:
        return round(xirr(transaction_dates, cash_flows), 4)
    except:
        return 'N/A'

# def calculate_portfolio_value(date, currency=None, asset_id=None, broker_id_list=None):
#     portfolio_value = 0

#     if asset_id is None:
#         portfolio_value = NAV_at_date(broker_id_list, date, currency, [])['Total NAV']
#     else:
#         asset = Assets.objects.get(id=asset_id)
#         # print(f"utils.py. line 165. asset current price: {asset.current_price(date)}")
#         # print(f"utils.py. line 167. Asset data: {asset.current_price(date).price}, {asset.position(date, broker_id_list)}")
#         portfolio_value = round(asset.price_at_date(date, currency).price * asset.position(date, broker_id_list), 2)

#     return portfolio_value

security_id = 4
security = Assets.objects.get(id=security_id)
print('Price:', security.price_at_date(effective_current_date).price)
print('Value:', calculate_portfolio_value(effective_current_date, security.currency, security_id))

Irr(effective_current_date, security.currency, security_id)

Price: 165.00
Value: 16500.00
utils.py. line 127. Transaction details: 50.00
utils.py. line 127. Transaction details: None
utils.py. line 127. Transaction details: 50.00
utils. line 146. Cash flows: [Decimal('-5000.36'), Decimal('0.76'), Decimal('-7500.50'), Decimal('16500.00')], Transaction dates: [datetime.date(2020, 6, 2), datetime.date(2020, 6, 25), datetime.date(2020, 9, 1), datetime.date(2024, 5, 22)]


0.0753

In [36]:
def calculate_effective_price(security, date, type='buy', currency=None, broker_id_list=None, start_date=None):
        """
        Calculates the buy-in price for the given date, currency, broker ID list, and start date.

        Args:
            date (datetime.date): The date for which to calculate the buy-in price.
            currency (str): The currency in which to calculate the buy-in price.
            broker_id_list (Optional[List[str]]): A list of broker IDs to filter the transactions by. Defaults to None.
            start_date (Optional[datetime.date]): The start date for the calculation. Defaults to None.

        Returns:
            float: The calculated buy-in price. Returns None if an error occurs.
        """
        
        try:

            is_long_position = None

            transactions = security.transactions.filter(
                quantity__isnull=False,  # Filter out transactions where quantity is not empty
                date__lte=date
            ).values('price', 'quantity', 'date', 'currency')
            
            if broker_id_list is not None:
                transactions = transactions.filter(broker_id__in=broker_id_list) 
            
            if not transactions:
                return None

            # Step 2: Get latest entry date
            entry_date = security.entry_dates(date, broker_id_list)[-1]
            # if self.position(date) == 0:
            #     entry_date = self.dates_of_zero_positions(date)[1]
            # else:
            #     entry_date = self.dates_of_zero_positions(date)[0]

            if start_date and start_date > entry_date:
                entry_date = start_date

                transactions = transactions.filter(date__gte=entry_date)
                position = security.position(entry_date, broker_id_list)
                if position != 0:
                    transactions = list(transactions) + [{
                        'price': security.price_at_date(entry_date).price,
                        'quantity': position,
                        'date': entry_date,
                        'currency': security.currency,
                    }]
                    is_long_position = position > 0
                print(transactions)
            else:
                transactions = transactions.filter(date__gte=entry_date)
                
            
            if is_long_position is None:
                first_transaction = transactions.order_by('date').first()
                is_long_position = first_transaction['quantity'] > 0

            # Step 3: Amend the calculation method. For every entry transaction buy-in price is the weighted average of previous buy-in price and price for the current transaction.
            value_entry = 0
            quantity_entry = 0
            value_exit = 0
            quantity_exit = 0

            # print(transactions)

            for transaction in transactions:
                
                if currency is not None:
                    fx_rate = FX.get_rate(transaction['currency'], currency, transaction['date'])['FX']
                else:
                    fx_rate = 1

                if fx_rate:
                    current_price = transaction['price'] * fx_rate
                    weight_current = transaction['quantity']

                    print("QE", quantity_entry)

                    # Calculate entry price
                    previous_entry_price = value_entry / quantity_entry if quantity_entry else 0
                    weight_entry_previous = quantity_entry
                    # If it's a long position and the quantity is positive, or if it's a short position and the quantity is negative, use the current price. Otherwise, use the previous buy-in price.
                    entry_price = current_price if (is_long_position and transaction['quantity'] > 0) or (not is_long_position and transaction['quantity'] < 0) else previous_entry_price
                    
                    if (weight_entry_previous + weight_current) == 0:
                        entry_price = previous_entry_price
                    else:
                        entry_price = (previous_entry_price * weight_entry_previous + entry_price * weight_current) / (weight_entry_previous + weight_current)
                    value_entry = entry_price * (quantity_entry + transaction['quantity'])
                    quantity_entry += transaction['quantity']

                    # Calculate exit price
                    previous_exit_price = value_exit / quantity_exit if quantity_exit else 0
                    weight_exit_previous = quantity_exit
                    # If it's a long position and the quantity is positive, or if it's a short position and the quantity is negative, use the current price. Otherwise, use the previous buy-in price.
                    exit_price = current_price if (is_long_position and transaction['quantity'] < 0) or (not is_long_position and transaction['quantity'] > 0) else previous_exit_price
                    
                    if (weight_exit_previous + weight_current) == 0:
                        exit_price = previous_exit_price
                    else:
                        exit_price = (previous_exit_price * weight_exit_previous + exit_price * weight_current) / (weight_exit_previous + weight_current)
                    value_exit = exit_price * (quantity_exit + transaction['quantity'])
                    quantity_exit += transaction['quantity']

                    print("QE2", quantity_entry)
            print(value_entry, quantity_entry, value_exit, quantity_exit)
            print(previous_entry_price)
            print(previous_exit_price)


            if type == 'entry':
                return value_entry / quantity_entry if quantity_entry else previous_entry_price
            elif type == 'exit':
                return value_exit / quantity_exit if quantity_exit else previous_exit_price

        except Exception as e:
            print(f"Error: {e}")
            return None
    
        

security_id = 7
security = Assets.objects.get(id=security_id)

calculate_effective_price(security, '2021-01-19', 'entry')

QE 0
QE2 -5.00
QE -5.00
QE2 -3.00
QE -3.00
QE2 0.00
0.0000 0.00 -0E-28 0.00
42.00
-24.66666666666666666666666667


Decimal('42.00')